# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)

X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 42)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
Y_pred = pipeline.predict(X_test)

In [10]:
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.62      0.58      0.60       986
          1       0.80      0.83      0.81      2031
          2       0.59      0.29      0.39        34

avg / total       0.74      0.74      0.74      3051

request
             precision    recall  f1-score   support

          0       0.80      0.93      0.86      1950
          1       0.83      0.58      0.68      1101

avg / total       0.81      0.80      0.80      3051

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3048
          1       0.00      0.00      0.00         3

avg / total       1.00      1.00      1.00      3051

aid_related
             precision    recall  f1-score   support

          0       0.78      0.91      0.84      1855
          1       0.82      0.60      0.69      1196

avg / total       0.80      0.79      0.78      3051

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 10)

In [12]:
final_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.29119258322798147, total=   8.3s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.7s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.3101559207753898, total=   8.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   21.3s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.27686472819216185, total=   8.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


...

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.32279814580699534, total=   7.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.31605562579013907, total=   7.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.31268436578171094, total=   7.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.3076274757690687, total=  16.1s
[CV] clf__estimator__min_samples_split=2, cl

[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5, score=0.3556679308891698, total=  13.8s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5, score=0.3333333333333333, total=  13.9s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, score=0.30299199325748, total=  14.2s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, score=0.31352718078381797, total=  14.3s
[CV] clf__estimator__min_samples_split=5, clf__es

[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5, score=0.3396544458491361, total=  12.4s


[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 16.1min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
Y_pred_final = final_model.predict(X_test)

In [15]:
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred_final[:, i]))

related
             precision    recall  f1-score   support

          0       0.68      0.57      0.62       986
          1       0.80      0.87      0.83      2031
          2       0.56      0.41      0.47        34

avg / total       0.76      0.77      0.76      3051

request
             precision    recall  f1-score   support

          0       0.84      0.91      0.87      1950
          1       0.82      0.69      0.75      1101

avg / total       0.83      0.83      0.83      3051

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3048
          1       0.00      0.00      0.00         3

avg / total       1.00      1.00      1.00      3051

aid_related
             precision    recall  f1-score   support

          0       0.82      0.89      0.85      1855
          1       0.80      0.69      0.74      1196

avg / total       0.81      0.81      0.81      3051

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

parameters2 = {'vect__min_df': [5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__max_depth': [7], 
              'clf__estimator__min_samples_split': [10],
              'clf__estimator__min_samples_leaf': [10]}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2, verbose = 10)

In [17]:
tuned_model = cv2.fit(X_train, Y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=True, vect__min_df=5, score=0.35145385587863465, total=   5.3s
[CV] clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.2s remaining:    0.0s


[CV]  clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=True, vect__min_df=5, score=0.35861778339654443, total=   5.3s
[CV] clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   14.5s remaining:    0.0s


[CV]  clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=True, vect__min_df=5, score=0.3822166034555415, total=   5.3s
[CV] clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=False, vect__min_df=5 


...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   28.6s remaining:    0.0s


[CV]  clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=False, vect__min_df=5, score=0.3645174884112937, total=   4.8s
[CV] clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   35.3s remaining:    0.0s


[CV]  clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=False, vect__min_df=5, score=0.36999578592498944, total=   4.8s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   42.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   42.1s finished


In [ ]:
Y_pred_tuned = tuned_model.predict(X_test)

In [ ]:
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred_tuned[:, i]))

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(tuned_model, open('ml_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.